In [1]:
import pandas as pd
#import numpy as np
#import math
import datetime
import plotly.graph_objs as go
#DB Details
connection_type = "mssql+pyodbc"
dataportal_db_user = "dataportal"
dataportal_db_password = "FatShamingMarc"
dataportal_db_ip = "10.8.4.35"
db_port = "1433"
db_prod_name = "dataportal_prod"
db_driver = "ODBC+DRIVER+17+for+SQL+Server"

In [2]:
def create_connection_string(connection_type, user, password, host, port, database_name, driver):
    return (
        "{connection_type}://{user}:{password}@{host}:{port}/{database_name}"
        "?driver={driver}").format(
        connection_type=connection_type,
        user=user,
        password=password,
        host=host,
        port=port,
        database_name=database_name,
        driver=driver)

In [3]:
connection_string = create_connection_string(connection_type,dataportal_db_user,dataportal_db_password,dataportal_db_ip,db_port,db_prod_name, db_driver)

In [4]:
# adding region column to grouped_df based on below dictionary
country_mappings= {
    'GLOBAL': 'GLOBAL',
    'CANADA': 'NAM',
    'MEXICO': 'NAM',
    'U.S.A.': 'NAM',
    'AUSTRIA': 'Europe',
    'BELARUS': 'Europe',
    'BELGIUM' : 'Europe',
    'BOSNIA-HERZEGOVINA' : 'Europe',
    'CROATIA' : 'Europe',
    'CZECH REPUBLIC' : 'Europe',
    'DENMARK' : 'Europe',
    'FINLAND' : 'Europe',
    'FRANCE' : 'Europe',
    'GERMANY' : 'Europe',
    'GREECE' : 'Europe',
    'HUNGARY' : 'Europe',
    'IRELAND' : 'Europe',
    'ITALY' : 'Europe',
    'LITHUANIA' : 'Europe',
    'MACEDONIA' : 'Europe',
    'NETHERLANDS' : 'Europe',
    'NORWAY' : 'Europe',
    'POLAND' : 'Europe',
    'PORTUGAL' : 'Europe',
    'ROMANIA' : 'Europe',
    'SERBIA' : 'Europe',
    'SLOVAKIA' : 'Europe',
    'SPAIN' : 'Europe',
    'SWEDEN' : 'Europe',
    'SWITZERLAND' : 'Europe',
    'UKRAINE' : 'Europe',
    'UNITED KINGDOM' : 'Europe',
    'RUSSIA' : 'Russia',
    'CHINA' : 'China',
    'BAHRAIN' : 'Middle East',
    'JORDAN' : 'Middle East',
    'KUWAIT' : 'Middle East',
    'IRAN' : 'Middle East',
    'IRAQ' : 'Middle East',
    'ISRAEL' : 'Middle East',
    'OMAN' : 'Middle East',
    'QATAR' : 'Middle East',
    'SAUDI ARABIA' : 'Middle East',
    'SYRIA' : 'Middle East',
    'UNITED ARAB EMIRATES' : 'Middle East',
    'YEMEN' : 'Middle East'
}

In [5]:
country_mappings_df = pd.DataFrame(list(country_mappings.items()), columns=['COUNTRY', 'REGION'])

In [6]:
region_list = country_mappings_df['REGION'].unique().tolist()

In [7]:
sql_query_1month = "select * from udf_data_iir_offlineevents_unique_latest(dateadd(month,-1,getdate()))"
sql_query_latest = "select * from udf_data_iir_offlineevents_unique_latest(getdate())"

In [8]:
#retrieves data using specific query
df_latest = pd.read_sql(sql_query_latest, connection_string)
df_1month = pd.read_sql(sql_query_1month, connection_string)

In [9]:
# creating multiindex df with each event as single timeseries
dates = pd.date_range(start='1970-01-01', end='2021-12-31', freq='D')

In [10]:
def concat_df_with_new_column_value(df, columns, new_value):
    df_copy = df.copy()
    df[columns]=new_value
    return pd.concat([df, df_copy])

def transform_to_combined_dateindex(df, dates):
    grouped_df = df.groupby(['EVENT_START', 'EVENT_END', 'COUNTRY', 'EVENT_TYPE', 'UNIT_TYPE_DESCRIPTION'])\
                            .agg({"CAPACITY_OFFLINE": "sum"}).reset_index();

    grouped_df = grouped_df.merge(country_mappings_df, how='left', left_on='COUNTRY', right_on='COUNTRY')
    grouped_df = concat_df_with_new_column_value(grouped_df, ['COUNTRY','REGION'], 'GLOBAL')
    grouped_df = concat_df_with_new_column_value(grouped_df, 'EVENT_TYPE', 'All')

    start_df = pd.pivot_table(grouped_df, index='EVENT_START', values='CAPACITY_OFFLINE',
                              columns=['REGION','EVENT_TYPE','UNIT_TYPE_DESCRIPTION'],
                              aggfunc='sum').reindex(dates).fillna(0)
    end_df = pd.pivot_table(grouped_df, index='EVENT_END', values='CAPACITY_OFFLINE',
                            columns=['REGION','EVENT_TYPE','UNIT_TYPE_DESCRIPTION'],
                            aggfunc='sum').reindex(dates).fillna(0) * -1

    comb_df = start_df.add(end_df).cumsum()

    comb_df_m = comb_df.resample('M').mean()
    comb_df_m['MONTH'] = comb_df_m.index.month
    comb_df_m['YEAR'] = comb_df_m.index.year
    comb_df_m = comb_df_m.set_index(['YEAR', 'MONTH'])
    return comb_df_m


In [11]:
comb_df_latest = transform_to_combined_dateindex(df_latest, dates)
comb_df_m_1month = transform_to_combined_dateindex(df_1month, dates)

In [12]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
years_2 = [2020,2019]
years_5 = [2016,2017,2018,2019,2020]
region = 'GLOBAL'
planning_type = 'All'
distillation_type = 'Atmospheric Distillation'

In [13]:
trace = []

# 2019 & 2020
for year in years_2:
    trace.append(go.Scatter(x= months,
                             y= comb_df_latest.loc[year,(region,planning_type,distillation_type)],
                             mode = 'lines + markers',
                             line_shape='spline',
                             name = year))
# Draw data as of 1 month ago
trace.append(go.Scatter(x= months,
                         y= comb_df_m_1month.loc[2020,(region,planning_type,distillation_type)],
                         mode = 'lines + markers',
                         line_shape='spline',
                         name = '1 month ago'))
# Draw 5 year average
trace.append(go.Scatter(x= months,
                         y= comb_df_latest.loc[years_5,(region,planning_type,distillation_type)].mean(axis=0, level='MONTH'),
                         mode = 'lines + markers',
                         line_shape='spline',
                         name = '5yr avg'))

# layout = go.Layout(
                  colorway=['#17becf','#e377c2','#ff7f0e','#2ca02c','black','grey'],
                  title = {'text': ' '},
                  margin={'t': 40, 'b': 40, 'l': 40, 'r': 10},
                  legend=dict(orientation='h', yanchor="bottom", y=-0.25, xanchor="left", x=0)
                   )
traces = [trace]
data = [val for sublist in traces for val in sublist]
# define figure for graph output
figure = go.Figure(data = data, layout = layout)
figure.show()